In [3]:
from sympy import *

In [4]:
# do: object distance
# di: image distance
theta, do, di, S = symbols('theta d_o d_i S', real=True, positive=True)

In [5]:
theta + di + S + do

S + d_i + d_o + theta

In [6]:
Rl = Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Rl # Rotating matrix from world frame to left camera frame

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [7]:
Rr = Rl
Rr # Rotating matrix from world frame to right camera frame

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [8]:
Tl = Matrix([S/2, 0, -do])
Tl # Translating matrix from world frame to left camera frame

Matrix([
[ S/2],
[   0],
[-d_o]])

In [9]:
Tr = Matrix([-S/2, 0, -do])
Tr # Translating matrix from world frame to right camera frame

Matrix([
[-S/2],
[   0],
[-d_o]])

In [10]:
x, y, z, dx, dy, dz = symbols('x y z Delta_x Delta_y Delta_z', real=True)

Xw1 = Matrix([x, y, z]) # World coordinates of the point
Xw2 = Matrix([x + dx, y + dy, z + dz]) # World coordinates of the point
flipZ = Matrix([[1, 0, 0], [0, 1, 0], [0, 0, -1]]) # Apply reflection matrix to flip Z direction between world frame and camera frame

In [11]:
extrinsicl = Rl.row_join(Tl) # Extrinsic matrix from world frame to left camera frame
extrinsicr = Rr.row_join(Tr) # Extrinsic matrix from world frame to right camera frame
Pr = flipZ * extrinsicr # Projection matrix from world frame to right camera frame
Pl = flipZ * extrinsicl # Projection matrix from world frame to left camera frame

In [12]:
Xl1 = Pl * Xw1.row_insert(3, Matrix([1])) # Point 1 in left camera frame
Xl2 = Pl * Xw2.row_insert(3, Matrix([1])) # Point 2 in left camera frame
Xr1 = Pr * Xw1.row_insert(3, Matrix([1])) # Point 1 in right camera frame
Xr2 = Pr * Xw2.row_insert(3, Matrix([1])) # Point 2 in right camera frame

In [ ]:
Xil1 = Xl1 / Xl1[2] * di # Point 1 in left image frame
Xil1.subs({S: 0, z: 0})

Matrix([
[d_i*x/d_o],
[d_i*y/d_o],
[      d_i]])

In [43]:
Xil2 = Xl2 / Xl2[2] * di # Point 2 in left image frame
Xil2.subs({S: 0, z: 0})

Matrix([
[d_i*(Delta_x + x)/(-Delta_z + d_o)],
[d_i*(Delta_y + y)/(-Delta_z + d_o)],
[                               d_i]])

In [48]:
dXil = Xil1 - Xil2 # Displacement vector in left image frame
simplify(dXil.subs({S: 0, z: 0})[0])

d_i*(Delta_x*d_o + Delta_z*x)/(d_o*(Delta_z - d_o))

In [13]:
Xol1  = do/Xl1[2] * Xl1 # Point 1 in left image frame object plane
Xol2  = do/Xl2[2] * Xl2 # Point 2 in left image frame object plane
Xor1  = do/Xr1[2] * Xr1 # Point 1 in right image frame object plane
Xor2  = do/Xr2[2] * Xr2 # Point 2 in right image frame object plane

In [14]:
Xol1

Matrix([
[d_o*(S/2 + x)/(d_o - z)],
[        d_o*y/(d_o - z)],
[                    d_o]])

In [15]:
Xol2

Matrix([
[d_o*(Delta_x + S/2 + x)/(-Delta_z + d_o - z)],
[      d_o*(Delta_y + y)/(-Delta_z + d_o - z)],
[                                         d_o]])

In [16]:
Xor1

Matrix([
[d_o*(-S/2 + x)/(d_o - z)],
[         d_o*y/(d_o - z)],
[                     d_o]])

In [17]:
Xor2

Matrix([
[d_o*(Delta_x - S/2 + x)/(-Delta_z + d_o - z)],
[      d_o*(Delta_y + y)/(-Delta_z + d_o - z)],
[                                         d_o]])

In [18]:
dXol = Xol2 - Xol1 # Distance in left image frame object plane
dXor = Xor2 - Xor1 # Distance in right image frame object plane

In [19]:
dXol 

Matrix([
[-d_o*(S/2 + x)/(d_o - z) + d_o*(Delta_x + S/2 + x)/(-Delta_z + d_o - z)],
[              -d_o*y/(d_o - z) + d_o*(Delta_y + y)/(-Delta_z + d_o - z)],
[                                                                      0]])

In [20]:
dXor

Matrix([
[-d_o*(-S/2 + x)/(d_o - z) + d_o*(Delta_x - S/2 + x)/(-Delta_z + d_o - z)],
[               -d_o*y/(d_o - z) + d_o*(Delta_y + y)/(-Delta_z + d_o - z)],
[                                                                       0]])

In [21]:
n = 3
dxil = symbols('dx_il1:%d' % (n+1)) # Distance in left image frame image plane
dxir = symbols('dx_ir1:%d' % (n+1)) # Distance in right image frame image plane
dxil, dxir

((dx_il1, dx_il2, dx_il3), (dx_ir1, dx_ir2, dx_ir3))

In [22]:
# Mn = di/do # Camera magnification
Mn = symbols('Mn')
sol_dx_dz = solve(
    (
        Eq(dXol[0], dxil[0] / (-Mn)), # 相似三角形法则，像平面和物平面距离比值等于像距和物距的比值，负号是因为翻转成像。
        Eq(dXor[0], dxir[0] / (-Mn)),
    ),
    (dx, dz)
)

In [23]:
sol_dx_dz

{Delta_x: (-S*d_o*dx_il1/2 - S*d_o*dx_ir1/2 + S*dx_il1*z/2 + S*dx_ir1*z/2 + d_o*dx_il1*x - d_o*dx_ir1*x - dx_il1*x*z + dx_ir1*x*z)/(Mn*S*d_o - d_o*dx_il1 + d_o*dx_ir1 + dx_il1*z - dx_ir1*z),
 Delta_z: (-d_o**2*dx_il1 + d_o**2*dx_ir1 + 2*d_o*dx_il1*z - 2*d_o*dx_ir1*z - dx_il1*z**2 + dx_ir1*z**2)/(Mn*S*d_o - d_o*dx_il1 + d_o*dx_ir1 + dx_il1*z - dx_ir1*z)}

In [24]:
sol_dx = sol_dx_dz[dx]
sol_dz = sol_dx_dz[dz]

In [25]:
sol_dy1 = solve(
    (Eq(dXol[1], dxil[1] / (-Mn)),),
    dy
)

sol_dy2 = solve(
    (Eq(dXor[1], dxir[1] / (-Mn)),),
    dy
)

sol_dy = (sol_dy1[dy] + sol_dy2[dy]) / 2

In [26]:
simplify(sol_dx.subs(z, 0))

(-S*dx_il1/2 - S*dx_ir1/2 + dx_il1*x - dx_ir1*x)/(Mn*S - dx_il1 + dx_ir1)

In [27]:
simplify(sol_dy.subs(z, 0))

(-2*Delta_z*Mn*y + Delta_z*dx_il2 + Delta_z*dx_ir2 - d_o*dx_il2 - d_o*dx_ir2)/(2*Mn*d_o)

In [28]:
simplify(sol_dz.subs(z, 0))

d_o*(-dx_il1 + dx_ir1)/(Mn*S - dx_il1 + dx_ir1)

In [29]:
print(latex(sol_dx_dz))
print(latex(sol_dy))

\left\{ \Delta_{x} : \frac{- \frac{S d_{o} dx_{il1}}{2} - \frac{S d_{o} dx_{ir1}}{2} + \frac{S dx_{il1} z}{2} + \frac{S dx_{ir1} z}{2} + d_{o} dx_{il1} x - d_{o} dx_{ir1} x - dx_{il1} x z + dx_{ir1} x z}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}, \  \Delta_{z} : \frac{- d_{o}^{2} dx_{il1} + d_{o}^{2} dx_{ir1} + 2 d_{o} dx_{il1} z - 2 d_{o} dx_{ir1} z - dx_{il1} z^{2} + dx_{ir1} z^{2}}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}\right\}
\frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{il2} + \Delta_{z} dx_{il2} z + d_{o}^{2} dx_{il2} - 2 d_{o} dx_{il2} z + dx_{il2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)} + \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{ir2} + \Delta_{z} dx_{ir2} z + d_{o}^{2} dx_{ir2} - 2 d_{o} dx_{ir2} z + dx_{ir2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)}


$$
\left\{ \Delta_{x} : \frac{- \frac{S d_{o} dx_{il1}}{2} - \frac{S d_{o} dx_{ir1}}{2} + \frac{S dx_{il1} z}{2} + \frac{S dx_{ir1} z}{2} + d_{o} dx_{il1} x - d_{o} dx_{ir1} x - dx_{il1} x z + dx_{ir1} x z}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}, \  \Delta_{z} : \frac{- d_{o}^{2} dx_{il1} + d_{o}^{2} dx_{ir1} + 2 d_{o} dx_{il1} z - 2 d_{o} dx_{ir1} z - dx_{il1} z^{2} + dx_{ir1} z^{2}}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}\right\}
$$

$$
\Delta_{y}: \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{il2} + \Delta_{z} dx_{il2} z + d_{o}^{2} dx_{il2} - 2 d_{o} dx_{il2} z + dx_{il2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)} + \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{ir2} + \Delta_{z} dx_{ir2} z + d_{o}^{2} dx_{ir2} - 2 d_{o} dx_{ir2} z + dx_{ir2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)}
$$